<a href="https://colab.research.google.com/github/innosoft21/workspace/blob/main/tensorflow_python2/1_2_2_Huber_Loss_hyperparameter_and_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Huber Loss hyperparameter and class

이 실습에서는 이전 Huber 손실 함수를 확장하고 손실 함수를 정의하는 데 하이퍼 파라미터를 포함할 수 있는 방법을 보여 줍니다. 

또한 Loss 클래스를 상속하여 사용자 지정 손실을 개체로 구현하는 방법에 대해서도 알아보겠습니다.

In [6]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
from tensorflow import keras

이전과 같이, 이 모델은 아래 xs와 ys에서 학습될 것입니다. 

여기서 관계는 =2µ-1이므로, 나중에 x=10을 테스트할 때, 

어떤 버전의 모델이 19에 가장 가까운 답을 얻었는지가 더 정확한 것으로 간주됩니다.

In [7]:
# inputs
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)

# labels
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.compile()의 손실 인수는 두 개의 매개 변수, 

즉 ground truth(y_true)와 모델 예측(y_pred)을 받아들이는 함수만 허용한다. 

튜닝할 수 있는 하이퍼 파라미터를 포함하려면 이 하이퍼 파라미터를 받아들이는 래퍼 함수를 정의할 수 있다.

In [8]:
# wrapper function that accepts the hyperparameter
def my_huber_loss_with_threshold(threshold):
  
    # function that accepts the ground truth and predictions
    def my_huber_loss(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
        
        return tf.where(is_small_error, small_error_loss, big_error_loss) 

    # return the inner function tuned by the hyperparameter
    return my_huber_loss

In [9]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=my_huber_loss_with_threshold(threshold=1.2))
model.fit(xs, ys, epochs=500,verbose=0)
print(model.predict([10.0]))

[[18.556591]]


In [10]:
from tensorflow.keras.losses import Loss

class MyHuberLoss(Loss):
  
    # initialize instance attributes
    def __init__(self, threshold=1):
        super().__init__()
        self.threshold = threshold

    # compute loss
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= self.threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = self.threshold * (tf.abs(error) - (0.5 * self.threshold))
        return tf.where(is_small_error, small_error_loss, big_error_loss)

In [11]:
model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss=MyHuberLoss(threshold=1.02))
model.fit(xs, ys, epochs=500,verbose=0)
print(model.predict([10.0]))

[[18.764887]]
